<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/Untitled142.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('return.csv')

# Remove rows where NSE Code is blank
df = df.dropna(subset=['NSE Code'])

# Function to calculate the composite score
def calculate_composite_score(row):
    # Weight the returns (you can adjust these weights)
    return (
        row['Return over 3months'] * 0.3 +
        row['Return over 6months'] * 0.3 +
        row['Return over 1year'] * 0.4
    )

# Function to calculate the volatility score
def calculate_volatility_score(row):
    returns = [row['Return over 3months'], row['Return over 6months'], row['Return over 1year']]
    return np.std(returns)

# Function to calculate the fundamental score
def calculate_fundamental_score(row):
    pe_score = 1 / row['Price to Earning'] if row['Price to Earning'] > 0 else 0
    ps_score = 1 / row['Price to Sales'] if row['Price to Sales'] > 0 else 0
    roce_score = row['Return on capital employed'] / 100 if pd.notnull(row['Return on capital employed']) else 0
    d_to_e=1 / row['Debt to equity'] if row['Debt to equity'] > 0 else 0
    # Combine the scores (you can adjust these weights)
    return pe_score * 0.2 + ps_score * 0.2 + roce_score * 0.2+ d_to_e*0.4

# Calculate scores
df['Composite_Score'] = df.apply(calculate_composite_score, axis=1)
df['Volatility_Score'] = df.apply(calculate_volatility_score, axis=1)
df['Fundamental_Score'] = df.apply(calculate_fundamental_score, axis=1)

# Calculate the final score
df['Final_Score'] = (
    df['Composite_Score'] * 0.4 +
    (1 / df['Volatility_Score']) * 0.2 +  # Lower volatility is better
    (1/df['Fundamental_Score']) * 0.4
)

# Rank the stocks based on the final score
df['Rank'] = df['Final_Score'].rank(ascending=False)

# Sort the dataframe by rank
df_ranked = df.sort_values('Rank')


# Save the full ranked list to a CSV file
df_ranked.to_csv('ranked_stocks.csv', index=False)

print("\nFull ranked list has been saved to 'ranked_stocks.csv'")
print(f"\nTotal number of stocks ranked: {len(df_ranked)}")


Full ranked list has been saved to 'ranked_stocks.csv'

Total number of stocks ranked: 397


In [17]:
import pandas as pd
import numpy as np
from scipy import stats

# Read the CSV file
df = pd.read_csv('return.csv')

# Remove rows where NSE Code is blank
df = df.dropna(subset=['NSE Code'])

# Function to calculate the composite score
def calculate_composite_score(row):
    return (
        row['Return over 3months'] * 0.3 +
        row['Return over 6months'] * 0.3 +
        row['Return over 1year'] * 0.4
    )

# Function to calculate the volatility score
def calculate_volatility_score(row):
    returns = [row['Return over 3months'], row['Return over 6months'], row['Return over 1year']]
    return np.std(returns)

# Function to calculate the fundamental score
def calculate_fundamental_score(row):
    pe_score = 1 / row['Price to Earning'] if row['Price to Earning'] > 0 else 0
    ps_score = 1 / row['Price to Sales'] if row['Price to Sales'] > 0 else 0
    roce_score = row['Return on capital employed'] / 100 if pd.notnull(row['Return on capital employed']) else 0

    # Add debt to equity ratio (lower is better)
    de_score = 1 / (1 + abs(row['Debt to equity'])) if pd.notnull(row['Debt to equity']) else 0

    # Combine the scores (you can adjust these weights)
    return pe_score * 0.25 + ps_score * 0.25 + roce_score * 0.25 + de_score * 0.25

# Calculate scores
df['Composite_Score'] = df.apply(calculate_composite_score, axis=1)
df['Volatility_Score'] = df.apply(calculate_volatility_score, axis=1)
df['Fundamental_Score'] = df.apply(calculate_fundamental_score, axis=1)

# Standardize scores using Z-score normalization
df['Composite_Score_Std'] = stats.zscore(df['Composite_Score'])
df['Volatility_Score_Std'] = -stats.zscore(df['Volatility_Score'])  # Negative because lower volatility is better
df['Fundamental_Score_Std'] = stats.zscore(df['Fundamental_Score'])

# Calculate the final score using standardized scores
df['Final_Score'] = (
    df['Composite_Score_Std'] * 0.5 +
    df['Volatility_Score_Std'] * 0.2 +
    df['Fundamental_Score_Std'] * 0.3
)

# Rank the stocks based on the final score
df['Rank'] = df['Final_Score'].rank(ascending=False)

# Sort the dataframe by rank
df_ranked = df.sort_values('Rank')

# Select columns for display
columns_to_display = [
    'Name', 'NSE Code', 'Industry', 'Current Price',
    'Return over 3months', 'Return over 6months', 'Return over 1year',
    'Composite_Score_Std', 'Volatility_Score_Std', 'Fundamental_Score_Std', 'Final_Score', 'Rank'
]

# Display the top 10 stocks
print(df_ranked[columns_to_display].head(10))

# Save the full ranked list to a CSV file
df_ranked[columns_to_display].to_csv('ranked_stocks.csv', index=False)

print("\nFull ranked list has been saved to 'ranked_stocks.csv'")
print(f"\nTotal number of stocks ranked: {len(df_ranked)}")

                 Name    NSE Code                               Industry  \
306         Redington   REDINGTON                                Trading   
155           H P C L   HINDPETRO                             Refineries   
74            C P C L  CHENNPETRO                             Refineries   
283   PG Electroplast        PGEL               Electronics - Components   
262     Neuland Labs.  NEULANDLAB  Pharmaceuticals - Indian - Bulk Drugs   
151     Gravita India     GRAVITA             Mining / Minerals / Metals   
40      Authum Invest        AIIL                  Finance & Investments   
144  Godfrey Phillips  GODFRYPHLP                             Cigarettes   
44            B P C L        BPCL                             Refineries   
175           I O C L         IOC                             Refineries   

     Current Price  Return over 3months  Return over 6months  \
306         198.52                -8.81                -0.67   
155         434.40                2